In [1]:
import pandas as pd
import numpy as np

## Load Dataset

In [2]:
df_artworks = pd.read_csv('./collection/Artworks.csv')
#Necesary to drop repited columns
df_artists = pd.read_csv('./collection/Artists.csv')

### Explore Arttwork dataset

In [3]:
columns = list(filter( lambda x : x not in list(df_artists.columns), list(df_artworks.columns)))
df_artworks = df_artworks[columns]

In [4]:
df_artworks.head()

,Title,Artist,Date,Medium,Dimensions,CreditLine,AccessionNumber,Classification,Department,DateAcquired,...,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
0,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",Otto Wagner,1896,Ink and cut-and-pasted painted pages on paper,"19 1/8 x 66 1/2"" (48.6 x 168.9 cm)",Fractional and promised gift of Jo Carole and ...,885.1996,Architecture,Architecture & Design,1996-04-09,...,http://www.moma.org/media/W1siZiIsIjU5NDA1Il0s...,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN,NaN
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,1987,Paint and colored pencil on print,"16 x 11 3/4"" (40.6 x 29.8 cm)",Gift of the architect in honor of Lily Auchinc...,1.1995,Architecture,Architecture & Design,1995-01-17,...,http://www.moma.org/media/W1siZiIsIjk3Il0sWyJw...,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN,NaN
2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,1903,"Graphite, pen, color pencil, ink, and gouache ...","13 1/2 x 12 1/2"" (34.3 x 31.8 cm)",Gift of Jo Carole and Ronald S. Lauder,1.1997,Architecture,Architecture & Design,1997-01-15,...,http://www.moma.org/media/W1siZiIsIjk4Il0sWyJw...,NaN,NaN,NaN,34.3000,NaN,NaN,31.8000,NaN,NaN
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,1980,Photographic reproduction with colored synthet...,"20 x 20"" (50.8 x 50.8 cm)",Purchase and partial gift of the architect in ...,2.1995,Architecture,Architecture & Design,1995-01-17,...,http://www.moma.org/media/W1siZiIsIjEyNCJdLFsi...,NaN,NaN,NaN,50.8000,NaN,NaN,50.8000,NaN,NaN
4,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,1903,"Graphite, color pencil, ink, and gouache on tr...","15 1/8 x 7 1/2"" (38.4 x 19.1 cm)",Gift of Jo Carole and Ronald S. Lauder,2.1997,Architecture,Architecture & Design,1997-01-15,...,http://www.moma.org/media/W1siZiIsIjEyNiJdLFsi...,NaN,NaN,NaN,38.4000,NaN,NaN,19.1000,NaN,NaN


**Shape of artworks dataset**

In [5]:
df_artworks.shape

(138090, 23)

**Columns with NaN**

In [6]:
df_artworks.isnull().any()

Title                  True
Artist                 True
Date                   True
Medium                 True
Dimensions             True
CreditLine             True
AccessionNumber       False
Classification        False
Department            False
DateAcquired           True
Cataloged             False
ObjectID              False
URL                    True
ThumbnailURL           True
Circumference (cm)     True
Depth (cm)             True
Diameter (cm)          True
Height (cm)            True
Length (cm)            True
Weight (kg)            True
Width (cm)             True
Seat Height (cm)       True
Duration (sec.)        True
dtype: bool

**Date distribution**

**Check differents types of date**

In [ ]:
dates = df_artworks['Date'].dropna()
#drop Unknown values 
dates = dates[(dates != 'n.d.') & (dates != 'Unknown')]
dates.shape

In [ ]:
#Fast way to obtain dates (could be extracting wrong dates)
data_creation = dates.str.extract(r'(\d{4})')
data_creation = data_creation.dropna()
data_creation = pd.to_numeric(data_creation[0])
data_creation = data_creation.rename('Date')
data_creation.head()

In [ ]:
data_creation.hist(bins=100)

In [ ]:
# e.g 1987
date_dddd = dates[dates.str.match(r'\d{4}$')]
print(date_dddd.shape)
date_dddd.head()

In [ ]:
dates = dates.drop(date_dddd.index)
dates.head()

In [ ]:
#e.g 1983-...
date_period = dates[dates.str.match(r'\d{4}[- –].+')]
print(date_period.shape)
date_period.head()

In [ ]:
dates = dates.drop(date_period.index)
dates.head()

In [ ]:
#e.g. c. 1934...
date_c = dates[dates.str.match(r'^c\. \d{4}.*')]
print(date_c.shape)
date_c.head()

In [ ]:
dates = dates.drop(date_c.index)
dates.head()

In [ ]:
# e.g. 1968.
date_dddd_point = dates[dates.str.match(r'^\d{4}\.$')]
print(date_dddd_point.shape)
date_dddd_point.head()

**Classification distribution**

In [ ]:
df_artworks['Classification'].value_counts().plot.bar()

**Cataloged distribution**

In [ ]:
df_artworks['Cataloged'].value_counts()

### Artwork image extraction

In [7]:
from bs4 import BeautifulSoup
import urllib
import os
import zipfile

In [8]:
base_url = 'https://www.moma.org/'
base_image_path = os.path.join(os.getcwd(), 'collection/images')

**Drop artworks without link**

In [9]:
df_artworks_download = df_artworks.dropna(subset=['URL'])
df_artworks_download.head()

,Title,Artist,Date,Medium,Dimensions,CreditLine,AccessionNumber,Classification,Department,DateAcquired,...,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
0,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",Otto Wagner,1896,Ink and cut-and-pasted painted pages on paper,"19 1/8 x 66 1/2"" (48.6 x 168.9 cm)",Fractional and promised gift of Jo Carole and ...,885.1996,Architecture,Architecture & Design,1996-04-09,...,http://www.moma.org/media/W1siZiIsIjU5NDA1Il0s...,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN,NaN
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,1987,Paint and colored pencil on print,"16 x 11 3/4"" (40.6 x 29.8 cm)",Gift of the architect in honor of Lily Auchinc...,1.1995,Architecture,Architecture & Design,1995-01-17,...,http://www.moma.org/media/W1siZiIsIjk3Il0sWyJw...,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN,NaN
2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,1903,"Graphite, pen, color pencil, ink, and gouache ...","13 1/2 x 12 1/2"" (34.3 x 31.8 cm)",Gift of Jo Carole and Ronald S. Lauder,1.1997,Architecture,Architecture & Design,1997-01-15,...,http://www.moma.org/media/W1siZiIsIjk4Il0sWyJw...,NaN,NaN,NaN,34.3000,NaN,NaN,31.8000,NaN,NaN
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,1980,Photographic reproduction with colored synthet...,"20 x 20"" (50.8 x 50.8 cm)",Purchase and partial gift of the architect in ...,2.1995,Architecture,Architecture & Design,1995-01-17,...,http://www.moma.org/media/W1siZiIsIjEyNCJdLFsi...,NaN,NaN,NaN,50.8000,NaN,NaN,50.8000,NaN,NaN
4,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,1903,"Graphite, color pencil, ink, and gouache on tr...","15 1/8 x 7 1/2"" (38.4 x 19.1 cm)",Gift of Jo Carole and Ronald S. Lauder,2.1997,Architecture,Architecture & Design,1997-01-15,...,http://www.moma.org/media/W1siZiIsIjEyNiJdLFsi...,NaN,NaN,NaN,38.4000,NaN,NaN,19.1000,NaN,NaN


In [10]:
df_artworks_download.shape

(79542, 23)

**Download artwork's images**

In [11]:
# w to write a zipfile || a to append to an existing file
zf = zipfile.ZipFile(os.path.join(base_image_path, 'image.zip'), 'a')
zf.write(base_image_path)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Duplicate name: 'root/notebooks/collection/images/'
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
def download_image(x):
    global zf
    try:
        html = urllib.request.urlopen(x['URL'])
        soup = BeautifulSoup(html)
        imgs = soup.findAll('img')
        image_url = base_url + imgs[0].get('src')
        path = os.path.join(base_image_path, str(x.name)+'.jpg')
        urllib.request.urlretrieve(image_url, path)
        zf.write(path)
        os.remove(path)
        return path
    except:
        return 'error at index ' + str(x.name)

In [ ]:
df_artworks_download['Image path'] = df_artworks_download.loc[30001:40000].apply(download_image, axis=1)

In [ ]:
zf.close()

In [ ]:
df_artworks['URL'][0]

In [ ]:
html = urllib.request.urlopen(df_artworks['URL'][0])
soup = BeautifulSoup(html)

In [ ]:
artwork_name = soup.findAll('span', {'class': 'work__short-caption__text--primary balance-text'})
artwork_name[0].text

In [ ]:
imgs = soup.findAll('img')

In [ ]:
image_url = base_url + imgs[0].get('src')

In [ ]:
urllib.request.urlretrieve(image_url, os.path.join(base_image_path, '0.jpg') )

In [ ]:
urllib.urlretrieve('/media/W1siZiIsIjU5NDA1Il0sWyJwIiwiY29udmVydCIsIi1yZXNpemUgMjAwMHgyMDAwXHUwMDNlIl1d.jpg?sha=7a282c9a6f26636d', os.path.basename('example'))